In [1]:
import twitter

In [2]:
api = twitter.Api(consumer_key='NWEMlDFaUUaMJXboGHoNyaPJB',
                 consumer_secret='VwI1iV1eZ20N5z4UbuSOpCqonoE344DnPFsJDW8rKVpitr6O4g',
                 access_token_key='1355881-cRRUr9TwNlIM7z0UunZDjHukJVWkOkvPulV2ZerB0p',
                 access_token_secret='heqkCnkvYVjpOGZbTC1dV6gk97qlSSoRGMnCyEhRTvLR0')


In [3]:
print(api.VerifyCredentials())

{"created_at": "Sat Mar 17 14:40:04 +0000 2007", "favourites_count": 2, "followers_count": 5, "friends_count": 41, "id": 1355881, "lang": "en", "name": "Joseph Batz", "profile_background_color": "9AE4E8", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme1/bg.png", "profile_image_url": "http://pbs.twimg.com/profile_images/573916308951060480/aNArrcH3_normal.jpeg", "profile_link_color": "89C9FA", "profile_sidebar_fill_color": "E0FF92", "profile_text_color": "000000", "protected": true, "screen_name": "jbatz", "status": {"created_at": "Fri Mar 11 21:27:25 +0000 2016", "id": 708403982708813824, "id_str": "708403982708813824", "lang": "en", "source": "<a href=\"http://twitter.com\" rel=\"nofollow\">Twitter Web Client</a>", "text": "Flying with python: https://t.co/Hb1rBbYeHt"}, "statuses_count": 12, "time_zone": "Pacific Time (US & Canada)", "utc_offset": -28800}


In [14]:
def createTestData(search_string):
    try:
        tweets_fetched=api.GetSearch(search_string, count=100)
        # This will return a list with twitter.Status objects. These have attributes for 
        # text, hashtags etc of the tweet that you are fetching. 
        # The full documentation again, you can see by typing pydoc twitter.Status at the 
        # command prompt of your terminal 
        print("Great! We fetched "+str(len(tweets_fetched))+" tweets with the term "+search_string+"!!")
        # We will fetch only the text for each of the tweets, and since these don't have labels yet, 
        # we will keep the label empty 
        return [{"text":status.text,"label":None} for status in tweets_fetched]
    except:
        print("Sorry there was an error!")
        return None
    
search_string=input("Hi there! What are we searching for today?")
testData=createTestData(search_string)


Hi there! What are we searching for today?trump
Great! We fetched 100 tweets with the term trump!!


In [15]:
testData

[{'label': None,
  'text': 'Brian Ross, the reporter who made a fraudulent live newscast about me that drove the Stock Market down 350 points (… https://t.co/Cbz6nDw2l8'},
 {'label': None,
  'text': 'AT&amp;T gave bonuses saying "Because of the tax bill!!" Trump said, "See? More money, more jobs. Tax bill!!!"\n\nATT now… https://t.co/BUbaMa7zoF'},
 {'label': None,
  'text': 'Jake Tapper of Fake News CNN just got destroyed in his interview with Stephen Miller of the Trump Administration. W… https://t.co/eIIiydV1hR'},
 {'label': None,
  'text': 'RT @Clauwild1: Pauta atual do jornalismo brasileiro:              \n1- Falar mal do Trump. 2-Inventar alguma mentira sobre a família Bolsona…'},
 {'label': None,
  'text': 'RT @DavidCornDC: Steele, to be clear, was not officially an FBI informant. Never on the payroll. Passed his reports to the FBI because he w…'},
 {'label': None,
  'text': 'RT @Bisquettet: @FoxNews @realDonaldTrump &lt;&lt;&lt;&lt;Was it Trump who got these numbers low or was i

In [20]:

###############################################################
# Step 2: Classify each of the 100 tweets as positive or negative
################################################################

# 2a. Download a corpus of tweets to use as training data
# We'll use Niek Sanders's Tweet Sentiment Corpus. He has kindly provided 5000+ labelled tweets
# WE can download a csv from his website with the tweets. But there is a catch, Twitter only allows 
# sharing of the tweet_id, so we'll have to fetch the text for each tweet id from the twitter API 

# We'll write a function that will read the csv we got from his website, for each tweet id in the 
# csv we'll download the tweet text and then write it back to another csv 

def createTrainingCorpus(corpusFile,tweetDataFile):
    import csv
    corpus=[]
    with open(corpusFile,'r') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',',quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id":row[2],"label":row[1],"topic":row[0]})
    # We now have a list with a dictionary for each row in Sanders's csv
    # Next let's iterate through that list and fetch the text for each tweet_id
    
    # If you try to download more than 180 tweets/15 mins, Twitter will rate limit you. So, use a delay
    # to avoid being rate limited. But this means it will take 10+ hours to download all 5000 tweets 
    # We'll show you the code to download all 5000 tweets, but for now, we'll work with a smaller corpus
    # so we won't have to wait 10 hours to see our code run :) 
    
    # To download the full corpus 
    import time 
    rate_limit=180
    sleep_time=900/180
    
    trainingData=[]
    for tweet in corpus:
        try:
            status=api.GetStatus(tweet["tweet_id"])
            #Returns a twitter.Status object 
            print("Tweet fetched" + status.text)
            tweet["text"]=status.text
            #tweet is a dictionary which already has tweet_id and label (positive/negative/neutral)
            # Add another attribute now, the tweet text 
            trainingData.append(tweet)
            time.sleep(sleep_time) # to avoid being rate limited 
        except: 
            continue
    # Once the tweets are downloaded write them to a csv, so you won't have to wait 40 hours 
    # every time you run this code :) 
    with open(tweetDataFile,'wb') as csvfile:
        linewriter=csv.writer(csvfile,delimiter=',',quotechar="\"")
        for tweet in trainingData:
            try:
                linewriter.writerow([tweet["tweet_id"],tweet["text"],tweet["label"],tweet["topic"]])
            except Exception as e:
                print(e)
    return trainingData


In [21]:
def createLimitedTrainingCorpus(corpusFile,tweetDataFile):
    import csv
    corpus=[]
    with open(corpusFile,'r') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',',quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id":row[2],"label":row[1],"topic":row[0]})
    # We now have a list with a dictionary for each row in Sanders's csv
    # Next let's iterate through that list and fetch the text for each tweet_id
    
    # If you try to download more than 180 tweets/15 min, Twitter will rate limit you. So, use a delay
    # to avoid being rate limited. But this means it will take 10+ hours to download all 5000 tweets 
    # We'll show you the code to download all 5000 tweets, but for now, we'll work with a smaller corpus
    # so we won't have to wait 10 hours to see our code run :) 
    
    # To download the full corpus 
    
    trainingData=[]
    for label in ["positive","negative"]:
        i=1
        for tweet in corpus:
            if tweet["label"]==label and i<=50:
                try:
                    status=api.GetStatus(tweet["tweet_id"])
                    #Returns a twitter.Status object 
                    print("Tweet fetched" + status.text)
                    tweet["text"]=status.text
                    #tweet is a dictionary which already has tweet_id and label (positive/negative/neutral)
                    # Add another attribute now, the tweet text 
                    trainingData.append(tweet)
                    i=i+1
                except Exception as e: 
                    print(e)
                    
    # Once the tweets are downloaded write them to a csv, so you won't have to wait 10 hours 
    # every time you run this code :) 
    with open(tweetDataFile,'w') as csvfile:
        linewriter=csv.writer(csvfile,delimiter=',',quotechar="\"")
        # We'll add a try catch block here so that we still get the training data even if the write 
        # fails 
        for tweet in trainingData:
            try:
                linewriter.writerow([tweet["tweet_id"],tweet["text"],tweet["label"],tweet["topic"]])
            except Exception as e:
                print(e)
    return trainingData

corpusFile="corpus.csv"
tweetDataFile="tweetDataFile.csv"

trainingData=createLimitedTrainingCorpus(corpusFile,tweetDataFile)
# This will have saved our 150 tweets to a file and also returned a list with all the tweet data we 
# need for training


Tweet fetchedNow all @Apple has to do is get swype on the iphone and it will be crack. Iphone that is
[{'code': 179, 'message': 'Sorry, you are not authorized to see this status.'}]
Tweet fetchedHilarious @youtube video - guy does a duet with @apple 's Siri. Pretty much sums up the love affair! http://t.co/8ExbnQjY
Tweet fetched@RIM you made it too easy for me to switch to @Apple iPhone. See ya!
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
Tweet fetchedThe 16 strangest things Siri has said so far. I am SOOO glad that @Apple gave Siri a sense of humor! http://t.co/TWAeUDBp via @HappyPlace
Tweet fetchedGreat up close & personal event @Apple tonight in Regent St store!
Tweet fetchedFrom which companies do you experience the best customer service aside from @zappos and @apple?
Tweet fetchedJust apply for a job at @Apple, hope they call me lol
[{'code': 144, 'message': 'No status found with that ID.'}]
Tweet fetchedL

Tweet fetchedAsked siri is she dreams of electric sleep. Was disappointed that she didn't have a snippy answer.  Missed opportunity @apple
Tweet fetched@paulens It surprises me that @Apple throws up an error alert about authorizing, and there's no "Authorize this computer" button.
[{'code': 144, 'message': 'No status found with that ID.'}]
Tweet fetchedFUCK YOU @apple DIE IN A FUCKING BLAZE INFERNO.
Tweet fetchedOh, @apple. Steve obviously had nothing to do with iPhoto, as it's the perfect opposite of 'insanely great'. Get it fixed, please.
[{'code': 179, 'message': 'Sorry, you are not authorized to see this status.'}]
Tweet fetched@ryanbaldwin @apple So in iTunes I go Store -&gt; Authorise… why doesn't it just auto-authorise it when I sign into iTunes? Grrrr...
Tweet fetchedSeriously - I have absolutely no offing clue what @Apple means by "authorization", nor how to do it.
Tweet fetchedBoy, could @apple make it any harder to put my purchased music from the cloud on to my new macbook p

In [22]:
# 2b. A class to preprocess all the tweets, both test and training
# We will use regular expressions and NLTK for preprocessing 
import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 

class PreProcessTweets:
    def __init__(self):
        self._stopwords=set(stopwords.words('english')+list(punctuation)+['AT_USER','URL'])
        
    def processTweets(self,list_of_tweets):
        # The list of tweets is a list of dictionaries which should have the keys, "text" and "label"
        processedTweets=[]
        # This list will be a list of tuples. Each tuple is a tweet which is a list of words and its label
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self,tweet):
        # 1. Convert to lower case
        tweet=tweet.lower()
        # 2. Replace links with the word URL 
        tweet=re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)     
        # 3. Replace @username with "AT_USER"
        tweet=re.sub('@[^\s]+','AT_USER',tweet)
        # 4. Replace #word with word 
        tweet=re.sub(r'#([^\s]+)',r'\1',tweet)
        # You can do further cleanup as well if you like, replace 
        # repetitions of characters, for ex: change "huuuuungry" to "hungry"
        # We'll leave that as an exercise for you on regular expressions
        tweet=word_tokenize(tweet)
        # This tokenizes the tweet into a list of words 
        # Let's now return this list minus any stopwords 
        return [word for word in tweet if word not in self._stopwords]
    
tweetProcessor=PreProcessTweets()
ppTrainingData=tweetProcessor.processTweets(trainingData)
ppTestData=tweetProcessor.processTweets(testData)

In [23]:
ppTestData

[(['brian',
   'ross',
   'reporter',
   'made',
   'fraudulent',
   'live',
   'newscast',
   'drove',
   'stock',
   'market',
   '350',
   'points',
   '…'],
  None),
 (['amp',
   'gave',
   'bonuses',
   'saying',
   '``',
   'tax',
   'bill',
   "''",
   'trump',
   'said',
   '``',
   'see',
   'money',
   'jobs',
   'tax',
   'bill',
   "''",
   'att',
   'now…'],
  None),
 (['jake',
   'tapper',
   'fake',
   'news',
   'cnn',
   'got',
   'destroyed',
   'interview',
   'stephen',
   'miller',
   'trump',
   'administration',
   'w…'],
  None),
 (['rt',
   'pauta',
   'atual',
   'jornalismo',
   'brasileiro',
   '1-',
   'falar',
   'mal',
   'trump',
   '2-inventar',
   'alguma',
   'mentira',
   'sobre',
   'família',
   'bolsona…'],
  None),
 (['rt',
   'steele',
   'clear',
   'officially',
   'fbi',
   'informant',
   'never',
   'payroll',
   'passed',
   'reports',
   'fbi',
   'w…'],
  None),
 (['rt',
   'lt',
   'lt',
   'lt',
   'lt',
   'trump',
   'got',
   'numbe

In [24]:
# 2c. Extract features and train your classifier

# We'll use two methods - Naive Bayes and Support Vector Machines 

import nltk 
# Naive Bayes Classifier - We'll use NLTK's built in classifier to perform the classification

# First build a vocabulary 
def buildVocabulary(ppTrainingData):
    all_words=[]
    for (words,sentiment) in ppTrainingData:
        all_words.extend(words)
    # This will give us a list in which all the words in all the tweets are present
    # These have to be de-duped. Each word occurs in this list as many times as it 
    # appears in the corpus 
    wordlist=nltk.FreqDist(all_words)
    # This will create a dictionary with each word and its frequency
    word_features=wordlist.keys()
    # This will return the unique list of words in the corpus 
    return word_features

# NLTK has an apply_features function that takes in a user-defined function to extract features 
# from training data. We want to define our extract features function to take each tweet in 
# The training data and represent it with the presence or absence of a word in the vocabulary 

def extract_features(tweet):
    tweet_words=set(tweet)
    features={}
    for word in word_features:
        features['contains(%s)' % word]=(word in tweet_words)
        # This will give us a dictionary , with keys like 'contains word1' and 'contains word2'
        # and values as True or False 
    return features 

# Now we can extract the features and train the classifier 
word_features = buildVocabulary(ppTrainingData)
trainingFeatures=nltk.classify.apply_features(extract_features,ppTrainingData)
# apply_features will take the extract_features function we defined above, and apply it it 
# each element of ppTrainingData. It automatically identifies that each of those elements 
# is actually a tuple , so it takes the first element of the tuple to be the text and 
# second element to be the label, and applies the function only on the text 

NBayesClassifier=nltk.NaiveBayesClassifier.train(trainingFeatures)
# We now have a classifier that has been trained using Naive Bayes

# Support Vector Machines 
from nltk.corpus import sentiwordnet as swn
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer 

# We have to change the form of the data slightly. SKLearn has a CountVectorizer object. 
# It will take in documents and directly return a term-document matrix with the frequencies of 
# a word in the document. It builds the vocabulary by itself. We will give the trainingData 
# and the labels separately to the SVM classifier and not as tuples. 
# Another thing to take care of, if you built the Naive Bayes for more than 2 classes, 
# SVM can only classify into 2 classes - it is a binary classifier. 

svmTrainingData=[' '.join(tweet[0]) for tweet in ppTrainingData]
# Creates sentences out of the lists of words 

vectorizer=CountVectorizer(min_df=1)
X=vectorizer.fit_transform(svmTrainingData).toarray()
# We now have a term document matrix 
vocabulary=vectorizer.get_feature_names()

# Now for the twist we are adding to SVM. We'll use sentiwordnet to add some weights to these 
# features 

swn_weights=[]

for word in vocabulary:
    try:
        # Put this code in a try block as all the words may not be there in sentiwordnet (esp. Proper
        # nouns). Look for the synsets of that word in sentiwordnet 
        synset=list(swn.senti_synsets(word))
        # use the first synset only to compute the score, as this represents the most common 
        # usage of that word 
        common_meaning =synset[0]
        # If the pos_Score is greater, use that as the weight, if neg_score is greater, use -neg_score
        # as the weight 
        if common_meaning.pos_score()>common_meaning.neg_score():
            weight=common_meaning.pos_score()
        elif common_meaning.pos_score()<common_meaning.neg_score():
            weight=-common_meaning.neg_score()
        else: 
            weight=0
    except: 
        weight=0
    swn_weights.append(weight)


# Let's now multiply each array in our original matrix with these weights 
# Initialize a list

swn_X=[]
for row in X: 
    swn_X.append(np.multiply(row,np.array(swn_weights)))
# Convert the list to a numpy array 
swn_X=np.vstack(swn_X)


# We have our documents ready. Let's get the labels ready too. 
# Lets map positive to 1 and negative to 2 so that everything is nicely represented as arrays 
labels_to_array={"positive":1,"negative":2}
labels=[labels_to_array[tweet[1]] for tweet in ppTrainingData]
y=np.array(labels)

# Let's now build our SVM classifier 
from sklearn.svm import SVC 
SVMClassifier=SVC()
SVMClassifier.fit(swn_X,y)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [25]:
# Step 2d: Run the classifier on the 100 downloaded tweets 

# First Naive Bayes 
NBResultLabels=[NBayesClassifier.classify(extract_features(tweet[0])) for tweet in ppTestData]

# Now SVM 
SVMResultLabels=[]
for tweet in ppTestData:
    tweet_sentence=' '.join(tweet[0])
    svmFeatures=np.multiply(vectorizer.transform([tweet_sentence]).toarray(),np.array(swn_weights))
    SVMResultLabels.append(SVMClassifier.predict(svmFeatures)[0])
    # predict() returns  a list of numpy arrays, get the first element of the array 
    # there is only 1 element and array

In [26]:
# Step 3 : Get the majority vote and print the sentiment 

if NBResultLabels.count('positive')>NBResultLabels.count('negative'):
    print("NB Result Positive Sentiment " + str(100*NBResultLabels.count('positive')/len(NBResultLabels))+"%")
else: 
    print("NB Result Negative Sentiment " + str(100*NBResultLabels.count('negative')/len(NBResultLabels))+"%")
    
if SVMResultLabels.count(1)>SVMResultLabels.count(2):
    print("SVM Result Positive Sentiment " + str(100*SVMResultLabels.count(1)/len(SVMResultLabels))+"%")
else: 
    print("SVM Result Negative Sentiment " + str(100*SVMResultLabels.count(2)/len(SVMResultLabels))+"%")


NB Result Positive Sentiment 82.0%
SVM Result Positive Sentiment 85.0%
